<a href="https://colab.research.google.com/github/asserhasan/machine-learning-/blob/main/dogs_dataset_hog_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure
import os
import numpy as np
import skimage
import skimage.io
from skimage import transform
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage import transform
from skimage import io, color, feature, exposure
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import cv2
import xml.etree.ElementTree as ET

In [ ]:
# Function to extract HOG features from an image and visualize it
from skimage import exposure
from skimage.feature import hog
def extract_hog_features(image):
    # Convert the image to grayscal
    if image.shape[-1] == 3:
            grayscale_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
            grayscale_img = image
    resized_img = cv2.resize(grayscale_img,(64,128), interpolation=cv2.INTER_AREA)
    # Calculate HOG features
    img_eq = exposure.equalize_hist(resized_img)
    hog_features, hog_image = hog(img_eq, orientations=9, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True)

    return hog_features , hog_image

In [ ]:
root_images="/content/drive/MyDrive/image_dataset/images"
root_annots="/content/drive/MyDrive/image_dataset/Annotation"

In [ ]:
def bounding_box(image ,b):
    bpath = root_annots+str("/"+b+"/"+str(image.split(".")[0]))
    tree = ET.parse(bpath)
    root = tree.getroot()
    objects = root.findall('object')
    for o in objects:
        bndbox = o.find('bndbox') # reading bound box
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

    return (xmin,ymin,xmax,ymax)

In [ ]:
# Path to the root folder of your dataset
dataset_path = root_images

# List all subdirectories (assuming each subdirectory corresponds to a class)
class_folders = [f.path for f in os.scandir(dataset_path) if f.is_dir()]
features_list=[]
labels_list=[]

from google.colab.patches import cv2_imshow
import cv2

# Loop through each class folder
for class_folder in class_folders:
    class_name = os.path.basename(class_folder)

    # Loop through each image in the class folder
    for image_filename in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_filename)

        # Load the image
        xmin , ymin ,xmax,ymax = bounding_box(image_filename,class_name)
        image = cv2.imread(image_path)
        cropped_img = image[ymin:ymax, xmin:xmax]

        # Extract HOG features and visualize
        hog_features, hog_image = extract_hog_features(cropped_img)

        # Display the original image and the HOG features
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(2, 2), sharex=True, sharey=True)

        ax1.axis('off')
        ax1.imshow(cropped_img, cmap=plt.cm.gray)

        ax2.axis('off')
        ax2.imshow(hog_image, cmap=plt.cm.gray)

        plt.show()

        # Append HOG features to the features list
        features_list.append(hog_features)

        # Append the label to the labels list
        labels_list.append(int(class_name))


In [ ]:
np.unique(labels_list)

array([1, 2, 3, 4, 5])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x_train , x_test , y_train , y_test = train_test_split(features_list , labels_list ,test_size = 0.33, random_state=12)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C= 0.1 ,penalty = 'l2',solver='liblinear', max_iter=1000)
History = model.fit(x_train,y_train)

In [ ]:
y_predict = model.predict(x_test)
print(model.n_iter_)

[9 8 8 8 8]


In [ ]:
predictions = model.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)

In [ ]:
print(accuracy)

0.4810606060606061


In [ ]:
print (np.shape(features_list))

(799, 3780)
